# CandleFetcher 동작 테스트

각 거래소별로 캔들 데이터를 조회하여 동작을 확인합니다.

In [1]:
import sys
from pathlib import Path
from datetime import datetime, timedelta
import pandas as pd

# src 경로 추가
sys.path.insert(0, str(Path.cwd() / 'src'))

from candle_data_manager.Core.ConnectionManager.ConnectionManager import ConnectionManager
from candle_data_manager.Core.Models.Symbol import Symbol
from candle_data_manager.Core.CandleFetcher.API.CandleFetcher.CandleFetcher import CandleFetcher

In [2]:
# ConnectionManager 초기화 (database_url 미제공 시 기본 MySQL 사용)
conn_manager = ConnectionManager()

# CandleFetcher 초기화
fetcher = CandleFetcher(conn_manager)

print("CandleFetcher 초기화 완료")

2026-01-19 17:45:19.645 | DEBUG    | candle_data_manager.Core.ConnectionManager.ConnectionManager:__init__:10 - ConnectionManager 초기화 시작
2026-01-19 17:45:19.647 | INFO     | candle_data_manager.Core.ConnectionManager.ConnectionManager:__init__:15 - database_url 미제공, 기본 MySQL 설정 사용: mysql+pymysql://root@localhost/candle_data_manager
2026-01-19 17:45:19.672 | INFO     | candle_data_manager.Core.ConnectionManager.ConnectionManager:__init__:36 - ConnectionManager 초기화 완료: pool_size=5, max_overflow=10


CandleFetcher 초기화 완료


In [3]:
# 헬퍼 함수: 가격 변환 (int → float)
def to_display(value):
    """Storage format (int × 10^8) → Display format (float)"""
    return value / 100000000

def display_candles(data, limit=5):
    """캔들 데이터를 보기 좋게 출력"""
    if not data:
        print("데이터 없음")
        return
    
    # DataFrame으로 변환
    df = pd.DataFrame(data[:limit])
    
    # 시간 변환
    df['datetime'] = df['timestamp'].apply(lambda x: datetime.fromtimestamp(x))
    
    # 가격 변환
    for col in ['open', 'high', 'low', 'close', 'volume']:
        df[col] = df[col].apply(to_display)
    
    # 컬럼 순서 조정
    df = df[['datetime', 'open', 'high', 'low', 'close', 'volume']]
    
    print(f"총 {len(data)}개 캔들 (최근 {limit}개 표시)")
    print(df.to_string(index=False))
    print()

## 1. Binance Spot (현물)

In [4]:
# Binance Spot - BTC/USDT 1시간봉 (최근 10개)
symbol_binance_spot = Symbol(
    archetype="CRYPTO",
    exchange="BINANCE",
    tradetype="SPOT",
    base="BTC",
    quote="USDT",
    timeframe="1h"
)

end_time = datetime.now()
start_time = end_time - timedelta(hours=10)

print("=" * 60)
print("Binance Spot - BTC/USDT 1h")
print("=" * 60)

try:
    data = fetcher.fetch(symbol_binance_spot, int(start_time.timestamp()), int(end_time.timestamp()))
    display_candles(data)
except Exception as e:
    print(f"오류 발생: {e}")

Binance Spot - BTC/USDT 1h
총 10개 캔들 (최근 5개 표시)
           datetime     open     high      low    close     volume
2026-01-19 08:00:00 95498.60 95498.60 93615.40 93673.14 1930.81880
2026-01-19 09:00:00 93673.14 93673.14 91910.20 92711.02 4221.82713
2026-01-19 10:00:00 92711.02 92875.68 92247.97 92586.65  910.87085
2026-01-19 11:00:00 92586.65 92672.00 92383.94 92566.80  452.44264
2026-01-19 12:00:00 92566.80 92776.83 92500.00 92690.09  556.40187



## 2. Binance Futures (선물)

In [5]:
# Binance Futures - BTC/USDT 1시간봉 (최근 10개)
symbol_binance_futures = Symbol(
    archetype="CRYPTO",
    exchange="BINANCE",
    tradetype="FUTURES",
    base="BTC",
    quote="USDT",
    timeframe="1h"
)

print("=" * 60)
print("Binance Futures - BTC/USDT 1h")
print("=" * 60)

try:
    data = fetcher.fetch(symbol_binance_futures, int(start_time.timestamp()), int(end_time.timestamp()))
    display_candles(data)
except Exception as e:
    print(f"오류 발생: {e}")

Binance Futures - BTC/USDT 1h
총 10개 캔들 (최근 5개 표시)
           datetime    open    high     low   close    volume
2026-01-19 08:00:00 95442.9 95443.0 93550.3 93614.0 23027.405
2026-01-19 09:00:00 93614.0 93614.1 91800.0 92658.3 35377.427
2026-01-19 10:00:00 92658.2 92825.6 92173.7 92542.2  7897.945
2026-01-19 11:00:00 92542.3 92639.4 92333.4 92516.7  3549.342
2026-01-19 12:00:00 92516.8 92748.4 92450.0 92656.6  3165.549



## 3. Upbit (업비트)

In [6]:
# Upbit - BTC/KRW 일봉 (최근 10개)
symbol_upbit = Symbol(
    archetype="CRYPTO",
    exchange="UPBIT",
    tradetype="SPOT",
    base="BTC",
    quote="KRW",
    timeframe="1d"
)

end_time_daily = datetime.now()
start_time_daily = end_time_daily - timedelta(days=10)

print("=" * 60)
print("Upbit - BTC/KRW 1d")
print("=" * 60)

try:
    data = fetcher.fetch(symbol_upbit, int(start_time_daily.timestamp()), int(end_time_daily.timestamp()))
    display_candles(data)
except Exception as e:
    print(f"오류 발생: {e}")

Upbit - BTC/KRW 1d
총 200개 캔들 (최근 5개 표시)
           datetime        open        high         low       close     volume
2025-07-04 18:00:00 148737000.0 149360000.0 146929000.0 147797000.0 819.644070
2025-07-05 18:00:00 147798000.0 148400000.0 147655000.0 148197000.0 289.920196
2025-07-06 18:00:00 148197000.0 149100000.0 147411000.0 148500000.0 574.677328
2025-07-07 18:00:00 148500000.0 149100000.0 147196000.0 148100000.0 832.294617
2025-07-08 18:00:00 148100000.0 148890000.0 147100000.0 148302000.0 670.292498



## 4. FDR - 한국 주식 (KRX)

In [7]:
# FDR - 삼성전자 일봉 (최근 10개)
symbol_krx = Symbol(
    archetype="STOCK",
    exchange="KRX",
    tradetype="SPOT",
    base="005930",  # 삼성전자
    quote="KRW",
    timeframe="1d"
)

print("=" * 60)
print("KRX - 삼성전자(005930) 1d")
print("=" * 60)

try:
    data = fetcher.fetch(symbol_krx, int(start_time_daily.timestamp()), int(end_time_daily.timestamp()))
    display_candles(data)
except Exception as e:
    print(f"오류 발생: {e}")

KRX - 삼성전자(005930) 1d
총 6개 캔들 (최근 5개 표시)
           datetime     open     high      low    close     volume
2026-01-12 09:00:00 141000.0 142000.0 136900.0 138800.0 26271134.0
2026-01-13 09:00:00 139800.0 140200.0 136900.0 137600.0 22384699.0
2026-01-14 09:00:00 137000.0 140300.0 136800.0 140300.0 18444394.0
2026-01-15 09:00:00 139000.0 144000.0 138300.0 143900.0 24701126.0
2026-01-16 09:00:00 145300.0 149500.0 144300.0 148900.0 30000219.0



## 5. FDR - 미국 주식 (NYSE)

In [8]:
# FDR - Apple (AAPL) 일봉 (최근 10개)
symbol_nyse = Symbol(
    archetype="STOCK",
    exchange="NYSE",
    tradetype="SPOT",
    base="AAPL",
    quote="USD",
    timeframe="1d"
)

print("=" * 60)
print("NYSE - Apple(AAPL) 1d")
print("=" * 60)

try:
    data = fetcher.fetch(symbol_nyse, int(start_time_daily.timestamp()), int(end_time_daily.timestamp()))
    display_candles(data)
except Exception as e:
    print(f"오류 발생: {e}")

NYSE - Apple(AAPL) 1d
총 6개 캔들 (최근 5개 표시)
           datetime       open       high        low      close     volume
2026-01-09 09:00:00 259.079987 260.209991 256.220001 259.369995 39997000.0
2026-01-12 09:00:00 259.160004 261.299988 256.799988 260.250000 45263800.0
2026-01-13 09:00:00 258.720001 261.809998 258.390015 261.049988 45730800.0
2026-01-14 09:00:00 259.489990 261.820007 256.709991 259.959991 40019400.0
2026-01-15 09:00:00 260.649994 261.040009 257.049988 258.209991 39388600.0



## 6. Provider 캐싱 확인

In [7]:
# 캐시된 Provider 목록 확인
print("=" * 60)
print("캐시된 Provider 목록")
print("=" * 60)

for key, provider in fetcher._providers.items():
    print(f"{key}: {type(provider).__name__}")

캐시된 Provider 목록
BINANCE_SPOT: BinanceSpotProvider
BINANCE_FUTURES: BinanceFuturesProvider
UPBIT: UpbitProvider


In [8]:
# 정리
conn_manager.close()
print("테스트 완료")

2026-01-19 17:47:31.548 | INFO     | candle_data_manager.Core.ConnectionManager.ConnectionManager:close:86 - 모든 DB 연결 종료


테스트 완료
